In [2]:
from utils import process_data
import pandas as pd
# 读取数据
df = pd.read_csv('../data/train.csv')

# 处理数据
processed_df = process_data(df)

# 查看处理后的列
print(processed_df.columns)
processed_df.head()

Index(['mid', 'eid', 'device_brand', 'ntt', 'operator', 'common_country',
       'common_province', 'common_city', 'appver', 'channel', 'os_type',
       'is_new_did', 'hour', 'day_of_week', 'is_weekend', 'days_since_first',
       'days_since_last', 'botId_code', 'pluginId_code'],
      dtype='object')


,mid,eid,device_brand,ntt,operator,common_country,common_province,common_city,appver,channel,os_type,is_new_did,hour,day_of_week,is_weekend,days_since_first,days_since_last,botId_code,pluginId_code
0,42,76,160,3,3,81,40,287,36,0,1,0,0.434783,0.833333,1,0.233333,0.225806,0.999876,0.0
1,31,116,65,3,0,81,79,438,69,5,0,1,0.391304,0.833333,1,0.466667,0.451613,0.999876,0.0
2,13,227,178,3,0,81,53,451,84,11,1,0,0.521739,0.333333,0,0.133333,0.129032,0.999876,0.0
3,21,52,28,3,1,81,195,123,54,12,1,1,0.086957,1.000000,1,0.733333,0.709677,0.999876,0.0
4,13,26,65,3,3,81,35,409,99,5,0,0,0.043478,0.333333,0,0.833333,0.806452,0.999876,0.0


In [ ]:
def analyze_did_transition(df, target_did=None):
    """
    分析同一个did随common_ts变化的is_new_did变化情况
    
    参数:
        df: 包含did, common_ts, is_new_did列的DataFrame
        target_did: 指定要分析的did（None则分析所有did）
    """
    # 确保必要的列存在
    required_cols = ['did', 'common_ts', 'is_new_did']
    if not all(col in df.columns for col in required_cols):
        raise ValueError(f"DataFrame需要包含这些列: {required_cols}")
    
    # 按did和common_ts排序
    df_sorted = df.sort_values(['did', 'common_ts'])
    
    # 筛选目标did或所有did
    if target_did is not None:
        df_sorted = df_sorted[df_sorted['did'] == target_did]
        if df_sorted.empty:
            print(f"没有找到did: {target_did}")
            return
    
    # 分组分析每个did的变化
    for did, group in df_sorted.groupby('did'):
        print(f"\n=== DID分析: {did} ===")
        print(f"记录数: {len(group)}")
        print(f"时间范围: {group['common_ts'].min()} 到 {group['common_ts'].max()}")
        
        # 打印时间序列变化
        print("\n时间序列变化:")
        print(group[['common_ts', 'is_new_did']].to_string(index=False))
        
        # 统计状态变化
        changes = (group['is_new_did'].diff() != 0).sum()
        print(f"\n状态变化次数: {changes}")
        
        # 初始和最终状态
        print(f"初始状态: {group['is_new_did'].iloc[0]}")
        print(f"最终状态: {group['is_new_did'].iloc[-1]}")
        
        # 如果有变化，打印变化点
        if changes > 0:
            change_points = group[group['is_new_did'].diff() != 0]
            print("\n状态变化点:")
            print(change_points[['common_ts', 'is_new_did']].to_string(index=False))

# 使用示例
# 假设df包含did, common_ts, is_new_did列
analyze_did_transition(df, target_did="0879243dfeddbeb27f35b16b39d1e5e502d9add9")  # 分析特定did


=== DID分析: 20b0bc2682226444068601a2d7b4a724d ===
记录数: 32
时间范围: 1740956012213 到 1743298121815

时间序列变化:
    common_ts  is_new_did
1740956012213           0
1740956730612           0
1741168892484           0
1741169150985           0
1741169151677           0
1741169460328           0
1741263234806           0
1741263374249           0
1741263374916           0
1741263700552           0
1741679297945           0
1741679384654           0
1741785428324           0
1741912579161           0
1741928376751           0
1741928377465           0
1743121403650           0
1743121477508           0
1743121483317           0
1743121653478           0
1743121666524           0
1743121668894           0
1743140690296           0
1743140690296           0
1743140691395           0
1743140697398           0
1743211054635           0
1743211129221           0
1743211271780           0
1743298002663           0
1743298036750           0
1743298121815           0

状态变化次数: 1
初始状态: 0
最终状态: 0

状态变化点:
    